# Preprocessing avec Scikit Learn

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from pandas import DataFrame, read_csv
from pprint import pprint
import seaborn as sns
from matplotlib import pyplot as plt

# Les methodes de preprocessing

fit transform

## SimpleImputer

Strategy
fill_value
missing_value

## OneHotEncoder

One hot encoder

## StandardScaler

## Autres?

# Pipelines

## Pipeline pour les features catégorielles

Pipeline prend en parametres une liste de tuples (nom, transform) qui définie une séquence de transformations.

In [ ]:
categorical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder())
])

- Les pipelines forment une arborescence de methodes et de paramêtres.
- Les pipelines exposent l'ensemble des paramètres.

In [ ]:
categorical_pipe

- Les paramètres sont préfixés par le nom donné dans le tuple.
- On peut utiliser get_param pour obtenir les paramètres.
- On peut utiliser set_param pour les modifier.
- Attention, il y'a 2 underscore!
- Chaque hyperparamètre pourra être optimisé.

In [ ]:
pprint(categorical_pipe.get_params())

In [ ]:
categorical_pipe.get_params()['imputer__strategy']

In [ ]:
categorical_pipe.set_params(imputer__strategy='constant', imputer__fill_value='NA')

## Pipeline pour les features numériques

In [ ]:
numeric_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

## ColumnTransformer

ColumnTransformer permet de combiner des traitements differents pour chaque groupe de colonnes.

In [ ]:
cat_columns=['Fence', 'CentralAir']
num_columns=['GrLivArea', 'LotFrontage', 'LotArea']

preprocess_pipe = ColumnTransformer([
    ('cat', categorical_pipe, cat_columns),
    ('num', numeric_pipe, num_columns)])


## Pipeline complet

Pipeline complet avec preprocessing et modèle

In [ ]:
model = LinearRegression()

full_pipe = Pipeline(
    [('pp', preprocess_pipe),
    ('reg', model)]
)

Demo sur un dataset?

In [ ]:
train_df = read_csv('./train.csv')
test_df = read_csv('./test.csv')
X = train_df[['GrLivArea', 'LotFrontage', 'LotArea', 'Fence', 'CentralAir']]
y = train_df[['SalePrice']]
X.columns

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

En une seule ligne:
- Preprocessing des features numériques et catégorielles
- Entrainement du modèle

In [ ]:
full_pipe.fit(X_train, y_train)

- Preprocessing
- Prediction sur le validation set et le test set

In [ ]:
y_train_pred = full_pipe.predict(X_train)
y_valid_pred = full_pipe.predict(X_valid)

In [ ]:
df = DataFrame(data={'value':y_valid['SalePrice'], 'predicted':y_valid_pred.reshape(-1,)}, index=y_valid.index)
plt.figure(figsize=(15,10))
sns.scatterplot(x='value',y='predicted', data=df )
plt.show()